### Code for simulations.

Needs two inputs: number of realisations of the simulation (num_sim) and the percentage of coverage (num_cov), from 0 to 4 --  (0%),1(25%),2(50%),3(75%),4(100%).

In [1]:
#!/usr/bin/env python
############ packages
from __future__ import division
from subprocess import call
import scipy as sp
from scipy.linalg import eigh as largest_eigh
from scipy.integrate import odeint
import numpy as np
import random
from itertools import combinations
from itertools import repeat
#import matplotlib.pyplot as plt
import math
import io
import csv
import sys, os
#import pandas as pd


In [2]:

num_sim = int(sys.argv[1])
num_cov = int(sys.argv[2])

#### CONDITIONS

cov_vector = [0,0.25,0.5,0.75,1]
num_humans = 100000
cycles = 10
num_types = 4
days_1cycle = 37
init_n = 10
init_freq = [0.05,0.05,0.05,0.85]
size_pool = np.copy(init_n * num_humans)

num_treated = np.int(num_humans * cov_vector[num_cov])
num_infected = num_humans * 0.1


###### PARAMETERS

b1 = 8 # growth rate exoerythrocytic phase
b2= 16 # growth rate erythrocytic phase
b4 = 2 #growth rate in mosquito
t_exoerythro = 5 # days
t_erythro = 8 ## or t exo plus 4, to see
t_gamete_form = 8
t_gamete_inf = 2
t_sporozoites = 10


###### ARTIFACTS

matrix = np.zeros((size_pool/init_n,num_types))
track_pool_freq = np.zeros((cycles+1,num_types))

total_counter = np.zeros(2)




ValueError: invalid literal for int() with base 10: '-f'

In [ ]:
##### LIFE CYCLE FUNCTIONS

def h_exoerythro(n):
    n_ts = np.array([n])
    ##n_ts_all[0,:,k] = np.copy(n)
    for i in range(0,num_types):
        n[i] = np.array(max(0,n[i] * (b1 - ((1 - w1[i])*2*b1))))
        n[i] = np.array(max(0,n[i] * (b1 - ((1 - w1[i])*2*b1))))
        n[i] = np.array(max(0,n[i] * (b1 - ((1 - w1[i])*2*b1))))
        n[i] = np.array(max(0,n[i] * (b1 - ((1 - w1[i])*2*b1))))
        n[i] = np.array(max(0,n[i] * (b1 - ((1 - w1[i])*2*b1))))
  ## n cannot be less than 0
    #n_ts_all[1+t,:,k] = np.copy(n)
    n_ts = np.vstack([n_ts,n])
    return(n,n_ts)

def h_erythro(n):
    n_ts = np.array([n])

    for i in range(0,num_types):
        n[i] = np.array(max(0,n[i] * (b2 - ((1 - w2[i])*2*b2))))
        n[i] = np.array(max(0,n[i] * (b2 - ((1 - w2[i])*2*b2))))
        n[i] = np.array(max(0,n[i] * (b2 - ((1 - w2[i])*2*b2))))
        n[i] = np.array(max(0,n[i] * (b2 - ((1 - w2[i])*2*b2))))
        #n_ts_all[t_exoerythro+t+1,:,k] = np.copy(n)
        #n_ts_all[t_exoerythro+t+2,:,k] = np.copy(n) ## because it happens every two time-steps, so we copy the value twice.
    n_ts = np.vstack([n_ts,n])
    return(n,n_ts)

def gamete_formation(n):
    n_ts = np.array([n])
    n = n * 0.0048
    n = n.astype(int)
    for t in range(0,t_gamete_form):
        #n_ts_all[t_exoerythro+t_erythro+t+1,:,k] = np.copy(n)
        n_ts = np.vstack([n_ts,n])
    return(n,n_ts)

def gamete_sex_bottleneck(n):
    n_ts = np.array([n])
    n_ts = np.vstack([n_ts,n])
    ## sex diff in human, before bottleneck
    females_n = np.copy(n) * 0.8
    females_n = females_n.astype(int)
    males_n = np.copy(n) * 0.2
    males_n = males_n.astype(int)

    n_total_females = np.sum(females_n)
    n_total_males = np.sum(males_n)
    n_total = n_total_females + n_total_males

    if n_total_females == 0 or n_total_males == 0: ## it won't be able to reproduce
        n = np.zeros(4)
        new_n = np.zeros(8)
        n_ts = np.vstack([n_ts,n])
    else:
        ## bottleneck
        freq_gametes = np.array((females_n/n_total, males_n/n_total))
        freq_reshape = np.reshape(freq_gametes,(1,2*num_types))
        freq_flatten = np.ndarray.flatten(freq_reshape)
        n_gametes = (24 * 2)

        new_n = np.random.multinomial(n_gametes,freq_flatten) ## this is the new n, but it has shape (1,8) bc the n types also have the feature sex becoming 8 types, so we store it as new_n

        for i in range(0,num_types):
            n[i] = new_n[i] + new_n[i+num_types] #sum of both male and female (in the new_n vector)for each genotype
        #n_ts_all[t_exoerythro+t_erythro+t_gamete_form+t_gamete_inf,:,k] = np.copy(n)
        n_ts = np.vstack([n_ts,n])
    return(n,new_n,n_ts)

def fecundation(new_n):
    n_total_females = np.sum(new_n[0:num_types])
    n_total_males = np.sum(new_n[num_types:(2*num_types)])
    zygots_matrix = np.zeros((num_types,num_types))
    if n_total_females != 0 or n_total_males != 0:
        #n_ts_all[t_exoerythro+t_erythro+t_gamete_form+t_gamete_inf+1,:,sim] = np.copy(n)

        while np.sum(new_n[4:8]) > 0:
            x1 = np.random.choice(np.arange(0,4))
            x2 = np.random.choice(np.arange(4,8))
            if new_n[x1] > 0 and new_n[x2] > 0:
                zygots_matrix[x1,x2-4] = zygots_matrix[x1,x2-4] + 1
                new_n[x1] = new_n[x1] - 1
                new_n[x2] = new_n[x2] - 1

    zygots_vector = [zygots_matrix[0,0]+ zygots_matrix[0,1],zygots_matrix[0,2]+ zygots_matrix[0,3]+ zygots_matrix[2,0]+ zygots_matrix[2,1],zygots_matrix[2,2]+ zygots_matrix[2,3],zygots_matrix[1,0]+ zygots_matrix[1,1],zygots_matrix[1,2]+ zygots_matrix[1,3]+ zygots_matrix[3,0]+ zygots_matrix[3,1],zygots_matrix[3,2]+ zygots_matrix[3,3]]


    return(zygots_vector)

def fecundation_viability_selection(zygots_vector):
    n[0] = 2 * zygots_vector[0] + 1 * zygots_vector[1]
    n[1] = 2 * zygots_vector[3] + 1 * zygots_vector[4]
    n[2] = 2 * zygots_vector[2] + 1 * zygots_vector[1]
    n[3] = 2 * zygots_vector[5] + 1 * zygots_vector[4]

    #n_ts_all[t_exoerythro+t_erythro+t_gamete_form+t_gamete_inf+1,:,k] = np.copy(n)

    fit = sp.random.binomial(1, w3_viables)
    zygots_vector = zygots_vector * (fit)

    n[0] = 2 * zygots_vector[0] + 1 * zygots_vector[1]
    n[1] = 2 * zygots_vector[3] + 1 * zygots_vector[4]
    n[2] = 2 * zygots_vector[2] + 1 * zygots_vector[1]
    n[3] = 2 * zygots_vector[5] + 1 * zygots_vector[4]

    #n_ts_all[t_exoerythro+t_erythro+t_gamete_form+t_gamete_inf+2,:,k] = np.copy(n)
    n_ts = np.array([n])
    return(n,n_ts)

def formation_sporozoites(n):
    n_ts = np.array([n])
    for i in range(0,num_types):
            n[i] = n[i] * (b4** 10) #* w4[i]
        #n_ts_all[t_exoerythro+t_erythro+t_gamete_form+t_gamete_inf+2+t+1,:,k] = np.copy(n)
    n_ts = np.vstack([n_ts,n])
    return(n,n_ts)

def final_sporozoites(n):
    n_ts = np.array([n])
    gametes_glands = np.copy(n) * 0.2
    gametes_glands.astype(int)
    if np.sum(gametes_glands) == 0:
        n = np.zeros(4)
    else:
        freq_glands = np.divide(gametes_glands,np.sum(gametes_glands))
        n = np.random.multinomial(init_n,freq_glands)

    #n_ts_all[t_exoerythro+t_erythro+t_gamete_form+t_gamete_inf+2+t_sporozoites+1,:,k] = np.copy(n)
    n_ts = np.vstack([n_ts,n])

    return(n,n_ts)




In [ ]:

#### CYCLES

# fitness matrices
w3_viables = [0.015,0.05,0.05,0.3,0.3,1]


## matrix initialization
for i in range(0,np.int(size_pool/init_n)):
    matrix[i]= np.random.multinomial(init_n,init_freq)

## counters initialization

track_ext = []
track_ext.append(np.zeros(2))
track_pool_size = []



## cycles

for c in range(0,cycles):
    # 1 individual cycle counters
    cycle_counter = np.zeros(2)
    k_ext = []
    inf_mosquito = 100000

    pool = np.sum(matrix,axis = 0) ### pool size splitted into genotypes
    track_pool_size.append(np.sum(pool)) ### track total pool size, independently of genotype

    x = random.sample(range(num_humans),num_treated)
    y = random.sample(range(num_humans),num_infected)

    if np.sum(pool) != 0: ## if it is 0, keep everything at 0
        pool_freq = pool/np.sum(pool)
        track_pool_freq[c]=np.copy(pool_freq)

        for k in range(0,num_humans):
            if k in x:
                #w2 = [1,1,0.1,0.1] # chloro
                w2 = [1,1,1,1]
                w1 = [1,0.1,1,0.1] # atov
            else:
                w2 = [1,1,1,1]
                # w1 = [1,0.1,1,0.1] # atov
                w1 = [1,1,1,1]

            n = np.copy(matrix[k])
            total_n = np.copy(np.sum(n))

            if total_n == 0 and k not in y: ## because the particular mosquito does not carry any parasite
                inf_mosquito = inf_mosquito - 1

            else:
                n,n_ts = h_exoerythro(n)
                n,n_ts = h_erythro(n)

                if np.sum(n) == 0: ## extinction bc of drug in the human
                    total_counter[0] = total_counter[0] + 1 ## general counter
                    cycle_counter[0] = cycle_counter[0] + 1 ## indiv cycle in humans
                else:
                    n,n_ts = gamete_formation(n)

                    if np.sum(n) == 0:
                       total_counter[0] = total_counter[0] + 1
                       cycle_counter[0] = cycle_counter[0] + 1 ##indiv cycle in humans
                    else:
                       n,new_n,n_ts = gamete_sex_bottleneck(n)

                       if np.sum(n) == 0:
                            total_counter[0] = total_counter[0] + 1
                            cycle_counter[0] = cycle_counter[0] + 1
                       else:
                            zygots_vector = fecundation(new_n)

                            if np.sum(zygots_vector) == 0:
                                n = np.zeros(4)
                                total_counter[1] = total_counter[1] + 1
                                cycle_counter[1] = cycle_counter[1] + 1
                            else:
                                n,n_ts = fecundation_viability_selection(zygots_vector)

                                if np.sum(n) == 0:
                                    total_counter[1] = total_counter[1] + 1
                                    cycle_counter[1] = cycle_counter[1] + 1
                                else:
                                    n, n_ts = formation_sporozoites(n)
                                    n, n_ts = final_sporozoites(n)


            
            matrix[k] = np.copy(n)
        track_ext.append(cycle_counter)
    else:
        track_ext.append(np.zeros(2))
        




final_pool = np.sum(matrix,axis = 0)

if np.sum(final_pool) != 0:
    final_pool_freq = final_pool/np.sum(final_pool)
    track_pool_freq[c+1]=np.copy(final_pool_freq)

track_pool_size.append(np.sum(final_pool))



np.savetxt("Freq_A_{0}_{1}.csv".format(num_sim,num_cov), track_pool_freq, delimiter=",") ## SERVES AS INPUT FOR 'GENOTYPE PLOTS'
np.savetxt("Ext_A_{0}_{1}.csv".format(num_sim,num_cov), track_ext, delimiter=",")
np.savetxt("Poolsize_A_{0}_{1}.csv".format(num_sim,num_cov), track_pool_size, delimiter=",")
